In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)

In [3]:
fa_feature_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

fa_feature_path = "/home/snu/Downloads/Histogram_AD_FA_20bin_updated.npy" # Assuming this is the AD data based on the request.
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])

num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (223, 180), Labels: (223,)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
from torch_geometric.nn import GATConv


class GAT_SemiSupervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device, heads=2, activ="SELU", dropout=0.25):
        super(GAT_SemiSupervised, self).__init__()
        self.device = device

        # GAT layer
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, concat=True, dropout=dropout)
        self.bn1 = nn.BatchNorm1d(hidden_dim * heads)  # heads multiply feature dimension if concat=True
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * heads, output_dim)

        # Activation mapping
        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu
        }
        self.act = activations.get(activ, nnFn.elu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GAT layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.fc(x)
        return logits

    def modularity_loss(self, A, logits):
        S = nnFn.softmax(logits, dim=1)
        d = torch.sum(A, dim=1)
        m = torch.sum(A)
        B = A - torch.outer(d, d) / (2 * m + 1e-12)

        modularity_term = (-1 / (2 * m + 1e-12)) * torch.trace(S.T @ B @ S)
        # Collapse regularization
        I_S = torch.eye(S.shape[1], device=self.device)
        k = torch.norm(I_S)
        n = S.shape[0]
        collapse_reg = (torch.sqrt(k) / n) * torch.norm(torch.sum(S, dim=0), p='fro') - 1
        entropy_reg = -torch.mean(torch.sum(S * torch.log(S + 1e-9), dim=1))
        return modularity_term + 0.1 * collapse_reg + 0.01 * entropy_reg


In [28]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.8
feats_dim = num_feats
hidden_dim = 512
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 100
lambda_mod = 0.1 #0.01  # weight for modularity loss
# lambda_sup = 5

In [31]:
W = create_adj(X, alpha)
data = load_data(W, X).to(device)
A_tensor = torch.from_numpy(W).float().to(device)
print(data)

Data(x=[223, 180], edge_index=[2, 41689])


In [32]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
    print(f"\n=== Fold {fold} ===")


    cn_idx = np.where(y == 0)[0]
    ad_idx = np.where(y == 1)[0]

    sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=fold)
    cn_train_idx, _ = next(sss_class.split(X[cn_idx], y[cn_idx]))
    ad_train_idx, _ = next(sss_class.split(X[ad_idx], y[ad_idx]))

    cn_train = cn_idx[cn_train_idx]
    ad_train = ad_idx[ad_train_idx]
    train_idx_final = np.concatenate([cn_train, ad_train])
    np.random.shuffle(train_idx_final)

    print(f"Train CN: {len(cn_train)}, Train AD: {len(ad_train)}")

    train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
    y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)


    model = GAT_SemiSupervised(feats_dim, hidden_dim, num_classes, device, activ="SELU").to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()


    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
        loss_unsup = model.modularity_loss(A_tensor, logits)
        total_loss = loss_sup + lambda_mod * loss_unsup

        total_loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: "
                  f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                  f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")


    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # Probability for class 1

    y_test = y[test_idx_global]
    y_pred_test = preds[test_idx_global]
    y_prob_test = probs[test_idx_global]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")


=== Fold 1 ===
Train CN: 13, Train AD: 9
Fold 1 Epoch 1: TotalLoss=0.601639 | Sup=0.615329 | Unsup=-0.136896 | TrainAcc=0.6818
Fold 1 Epoch 100: TotalLoss=0.435646 | Sup=0.450937 | Unsup=-0.152917 | TrainAcc=0.8182
Fold 1 Epoch 200: TotalLoss=0.383876 | Sup=0.399798 | Unsup=-0.159224 | TrainAcc=0.8182
Fold 1 Epoch 300: TotalLoss=0.097335 | Sup=0.113358 | Unsup=-0.160232 | TrainAcc=1.0000
Fold 1 Epoch 400: TotalLoss=0.016983 | Sup=0.033122 | Unsup=-0.161387 | TrainAcc=1.0000
Fold 1 Epoch 500: TotalLoss=-0.008469 | Sup=0.007643 | Unsup=-0.161119 | TrainAcc=1.0000
Fold 1 Epoch 600: TotalLoss=-0.012156 | Sup=0.004293 | Unsup=-0.164497 | TrainAcc=1.0000
Fold 1 Epoch 700: TotalLoss=-0.007775 | Sup=0.008530 | Unsup=-0.163048 | TrainAcc=1.0000
Fold 1 Epoch 800: TotalLoss=-0.015091 | Sup=0.001100 | Unsup=-0.161912 | TrainAcc=1.0000
Fold 1 Epoch 900: TotalLoss=-0.011840 | Sup=0.004765 | Unsup=-0.166054 | TrainAcc=1.0000
Fold 1 Epoch 1000: TotalLoss=-0.014845 | Sup=0.001516 | Unsup=-0.163613 | T

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# ==========================================
# CONFIG
# ==========================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_dim = 512
num_epochs = 2000
lr = 1e-4
weight_decay = 1e-4
batch_print_freq = 500  # print every 500 epochs

# Use same λ list as before
lambda_mod_list = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]



results_summary = []


for lambda_mod in lambda_mod_list:
    print(f"\n==============================")
    print(f" Running with λ_mod = {lambda_mod}")
    print(f"==============================")

    accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

    sss = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=SEED)

    for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
        print(f"\n=== Fold {fold} ===")

        cn_idx = np.where(y == 0)[0]
        ad_idx = np.where(y == 1)[0]

        sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=fold)
        cn_train_idx, _ = next(sss_class.split(X[cn_idx], y[cn_idx]))
        ad_train_idx, _ = next(sss_class.split(X[ad_idx], y[ad_idx]))

        cn_train = cn_idx[cn_train_idx]
        ad_train = ad_idx[ad_train_idx]
        train_idx_final = np.concatenate([cn_train, ad_train])
        np.random.shuffle(train_idx_final)

        print(f"Train CN: {len(cn_train)}, Train AD: {len(ad_train)}")

        train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
        y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)

        # Initialize model
        model = GCN_SemiSupervised(feats_dim, hidden_dim, num_classes, device, activ="RELU").to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        ce_loss = nn.CrossEntropyLoss()

        # Training loop
        for epoch in range(1, num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            logits = model(data)
            loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
            loss_unsup = model.modularity_loss(A_tensor, logits)
            total_loss = loss_sup + lambda_mod * loss_unsup

            total_loss.backward()
            optimizer.step()

            if epoch % batch_print_freq == 0 or epoch == 1:
                model.eval()
                with torch.no_grad():
                    preds_train = logits[train_idx_t].argmax(dim=1)
                    acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
                print(f"Fold {fold} Epoch {epoch}: "
                      f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                      f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

        # Evaluation
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out.argmax(dim=1).cpu().numpy()
            probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # probability for "AD"

        y_test = y[test_idx_global]
        y_pred_test = preds[test_idx_global]
        y_prob_test = probs[test_idx_global]

        acc = accuracy_score(y_test, y_pred_test)
        prec = precision_score(y_test, y_pred_test, zero_division=0)
        rec = recall_score(y_test, y_pred_test, zero_division=0)
        f1 = f1_score(y_test, y_pred_test, zero_division=0)
        auc = roc_auc_score(y_test, y_prob_test)
        ce = log_loss(y_test, y_prob_test)

        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        aucs.append(auc)
        ce_losses.append(ce)

        print(f"Fold {fold} → "
              f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
              f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")

    # Average results per λ_mod
    mean_acc, std_acc = np.mean(accuracies), np.std(accuracies)
    mean_prec, std_prec = np.mean(precisions), np.std(precisions)
    mean_rec, std_rec = np.mean(recalls), np.std(recalls)
    mean_f1, std_f1 = np.mean(f1_scores), np.std(f1_scores)
    mean_auc, std_auc = np.mean(aucs), np.std(aucs)
    mean_ce, std_ce = np.mean(ce_losses), np.std(ce_losses)

    results_summary.append({
        "λ_mod": lambda_mod,
        "Accuracy": f"{mean_acc:.4f} ± {std_acc:.4f}",
        "Precision": f"{mean_prec:.4f} ± {std_prec:.4f}",
        "Recall": f"{mean_rec:.4f} ± {std_rec:.4f}",
        "F1": f"{mean_f1:.4f} ± {std_f1:.4f}",
        "AUC": f"{mean_auc:.4f} ± {std_auc:.4f}",
        "CE Loss": f"{mean_ce:.4f} ± {std_ce:.4f}",
    })

    print(f"\n=== λ_mod = {lambda_mod} → Average Results ===")
    print(f"Accuracy:  {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision: {mean_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall:    {mean_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score:  {mean_f1:.4f} ± {std_f1:.4f}")
    print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
    print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")

# ==========================================
# Final summary table for all λ_mod values
# ==========================================
print("\n\n========== FINAL SUMMARY TABLE (CN vs AD) ==========")
results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False))


 Running with λ_mod = 0.001

=== Fold 1 ===
Train CN: 66, Train AD: 45
Fold 1 Epoch 1: TotalLoss=0.723331 | Sup=0.723465 | Unsup=-0.134165 | TrainAcc=0.3514
Fold 1 Epoch 500: TotalLoss=0.336209 | Sup=0.336367 | Unsup=-0.157570 | TrainAcc=0.8649
Fold 1 Epoch 1000: TotalLoss=0.247128 | Sup=0.247286 | Unsup=-0.158284 | TrainAcc=0.9189
Fold 1 Epoch 1500: TotalLoss=0.184764 | Sup=0.184924 | Unsup=-0.159232 | TrainAcc=0.9459
Fold 1 Epoch 2000: TotalLoss=0.143254 | Sup=0.143413 | Unsup=-0.159139 | TrainAcc=0.9550
Fold 1 → Acc=0.8125 | Prec=0.8000 | Rec=0.7111 | F1=0.7529 | AUC=0.8294 | CE Loss=0.7341

=== Fold 2 ===
Train CN: 66, Train AD: 45
Fold 2 Epoch 1: TotalLoss=0.742640 | Sup=0.742775 | Unsup=-0.134597 | TrainAcc=0.3333
Fold 2 Epoch 500: TotalLoss=0.384038 | Sup=0.384192 | Unsup=-0.153792 | TrainAcc=0.8468
Fold 2 Epoch 1000: TotalLoss=0.309953 | Sup=0.310107 | Unsup=-0.154670 | TrainAcc=0.8649
Fold 2 Epoch 1500: TotalLoss=0.245783 | Sup=0.245936 | Unsup=-0.153616 | TrainAcc=0.9279
Fol

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# ==========================================
# CONFIG
# ==========================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_dim = 512
num_epochs = 2000
lr = 1e-4
weight_decay = 1e-4
batch_print_freq = 500  # print every 500 epochs

# Use same λ list as before
lambda_mod_list = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]



results_summary = []


for lambda_mod in lambda_mod_list:
    print(f"\n==============================")
    print(f" Running with λ_mod = {lambda_mod}")
    print(f"==============================")

    accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

    sss = StratifiedShuffleSplit(n_splits=20, test_size=0.1, random_state=SEED)

    for fold, (train_val_idx, test_idx_global) in enumerate(sss.split(X, y), start=1):
        print(f"\n=== Fold {fold} ===")

        cn_idx = np.where(y == 0)[0]
        ad_idx = np.where(y == 1)[0]

        sss_class = StratifiedShuffleSplit(n_splits=20, test_size=0.1, random_state=fold)
        cn_train_idx, _ = next(sss_class.split(X[cn_idx], y[cn_idx]))
        ad_train_idx, _ = next(sss_class.split(X[ad_idx], y[ad_idx]))

        cn_train = cn_idx[cn_train_idx]
        ad_train = ad_idx[ad_train_idx]
        train_idx_final = np.concatenate([cn_train, ad_train])
        np.random.shuffle(train_idx_final)

        print(f"Train CN: {len(cn_train)}, Train AD: {len(ad_train)}")

        train_idx_t = torch.from_numpy(train_idx_final).long().to(device)
        y_train_tensor = torch.from_numpy(y[train_idx_final]).long().to(device)

        # Initialize model
        model = GCN_SemiSupervised(feats_dim, hidden_dim, num_classes, device, activ="RELU").to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        ce_loss = nn.CrossEntropyLoss()

        # Training loop
        for epoch in range(1, num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            logits = model(data)
            loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
            loss_unsup = model.modularity_loss(A_tensor, logits)
            total_loss = loss_sup + lambda_mod * loss_unsup

            total_loss.backward()
            optimizer.step()

            if epoch % batch_print_freq == 0 or epoch == 1:
                model.eval()
                with torch.no_grad():
                    preds_train = logits[train_idx_t].argmax(dim=1)
                    acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
                print(f"Fold {fold} Epoch {epoch}: "
                      f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                      f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

        # Evaluation
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out.argmax(dim=1).cpu().numpy()
            probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # probability for "AD"

        y_test = y[test_idx_global]
        y_pred_test = preds[test_idx_global]
        y_prob_test = probs[test_idx_global]

        acc = accuracy_score(y_test, y_pred_test)
        prec = precision_score(y_test, y_pred_test, zero_division=0)
        rec = recall_score(y_test, y_pred_test, zero_division=0)
        f1 = f1_score(y_test, y_pred_test, zero_division=0)
        auc = roc_auc_score(y_test, y_prob_test)
        ce = log_loss(y_test, y_prob_test)

        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1_scores.append(f1)
        aucs.append(auc)
        ce_losses.append(ce)

        print(f"Fold {fold} → "
              f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
              f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")

    # Average results per λ_mod
    mean_acc, std_acc = np.mean(accuracies), np.std(accuracies)
    mean_prec, std_prec = np.mean(precisions), np.std(precisions)
    mean_rec, std_rec = np.mean(recalls), np.std(recalls)
    mean_f1, std_f1 = np.mean(f1_scores), np.std(f1_scores)
    mean_auc, std_auc = np.mean(aucs), np.std(aucs)
    mean_ce, std_ce = np.mean(ce_losses), np.std(ce_losses)

    results_summary.append({
        "λ_mod": lambda_mod,
        "Accuracy": f"{mean_acc:.4f} ± {std_acc:.4f}",
        "Precision": f"{mean_prec:.4f} ± {std_prec:.4f}",
        "Recall": f"{mean_rec:.4f} ± {std_rec:.4f}",
        "F1": f"{mean_f1:.4f} ± {std_f1:.4f}",
        "AUC": f"{mean_auc:.4f} ± {std_auc:.4f}",
        "CE Loss": f"{mean_ce:.4f} ± {std_ce:.4f}",
    })

    print(f"\n=== λ_mod = {lambda_mod} → Average Results ===")
    print(f"Accuracy:  {mean_acc:.4f} ± {std_acc:.4f}")
    print(f"Precision: {mean_prec:.4f} ± {std_prec:.4f}")
    print(f"Recall:    {mean_rec:.4f} ± {std_rec:.4f}")
    print(f"F1-score:  {mean_f1:.4f} ± {std_f1:.4f}")
    print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
    print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")

# ==========================================
# Final summary table for all λ_mod values
# ==========================================
print("\n\n========== FINAL SUMMARY TABLE (CN vs AD) ==========")
results_df = pd.DataFrame(results_summary)
print(results_df.to_string(index=False))


 Running with λ_mod = 0.001

=== Fold 1 ===
Train CN: 119, Train AD: 81
Fold 1 Epoch 1: TotalLoss=0.718189 | Sup=0.718324 | Unsup=-0.134165 | TrainAcc=0.3500
Fold 1 Epoch 500: TotalLoss=0.399390 | Sup=0.399546 | Unsup=-0.155580 | TrainAcc=0.8150
Fold 1 Epoch 1000: TotalLoss=0.359025 | Sup=0.359182 | Unsup=-0.156118 | TrainAcc=0.8600
Fold 1 Epoch 1500: TotalLoss=0.299886 | Sup=0.300043 | Unsup=-0.156971 | TrainAcc=0.8750
Fold 1 Epoch 2000: TotalLoss=0.255661 | Sup=0.255818 | Unsup=-0.156252 | TrainAcc=0.8900
Fold 1 → Acc=0.9130 | Prec=0.8889 | Rec=0.8889 | F1=0.8889 | AUC=0.9603 | CE Loss=0.3024

=== Fold 2 ===
Train CN: 119, Train AD: 81
Fold 2 Epoch 1: TotalLoss=0.736928 | Sup=0.737062 | Unsup=-0.134597 | TrainAcc=0.3600
Fold 2 Epoch 500: TotalLoss=0.380164 | Sup=0.380319 | Unsup=-0.155516 | TrainAcc=0.8500
Fold 2 Epoch 1000: TotalLoss=0.329445 | Sup=0.329602 | Unsup=-0.156580 | TrainAcc=0.8700
Fold 2 Epoch 1500: TotalLoss=0.286209 | Sup=0.286364 | Unsup=-0.155606 | TrainAcc=0.8800
F